In [47]:
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras import regularizers
from keras import backend as K
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.layers import BatchNormalization
from keras.optimizers import SGD,Adam
from sklearn.metrics import mean_squared_error,r2_score
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense , Dropout
from keras.layers import LeakyReLU
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from scipy.sparse import csr_matrix
from sklearn.linear_model import OrthogonalMatchingPursuit
import datetime 
import scipy.io
import h5py

# directory_path = 'Data/Dataset_new/0dB/'
directory_path = '/kaggle/input/snr-data-1/Dataset_new2/different B_SNR_15/B 30/'

In [48]:
# A_mat = h5py.File(directory_path+ 'A.mat', 'r')
# A_3d = A_mat['A'][:]
# A_mat.close()

In [49]:
# A_3d.shape

In [50]:
# Psi1 = scipy.io.loadmat('/kaggle/input/snr-data-4/Dataset_new3/Psi1.mat')
# variable_names = Psi1.keys()
# print("Variable names in Psi1.mat:", variable_names)

# psim = Psi1['psim'][:]
# psim.shape

In [51]:
hs_mat = scipy.io.loadmat(directory_path+'hs_1.mat')
variable_names = hs_mat.keys()
print("Variable names in hs_mat.mat:", variable_names)

h_s = hs_mat['Xbar1'][:]
h_s.shape

Variable names in hs_mat.mat: dict_keys(['__header__', '__version__', '__globals__', 'Xbar1'])


(4096, 40000)

In [52]:
Yt_mat = scipy.io.loadmat(directory_path + 'Yt1.mat')
variable_names = Yt_mat.keys()
print("Variable names in Yt1.mat:", variable_names)
Yt_3d = Yt_mat['Yt'][:]
Yt_3d.shape

Variable names in Yt1.mat: dict_keys(['__header__', '__version__', '__globals__', 'Yt'])


(4096, 40000)

In [53]:
Yt_3d.shape

(4096, 40000)

In [54]:
h_s = h_s.T
h_s.shape

(40000, 4096)

In [55]:
Yt_3d = Yt_3d.T
Yt_3d.shape

(40000, 4096)

In [56]:
h_s.shape[0]

40000

In [57]:
c = 0
for i in range(0,h_s.shape[0]):
    y_test_csr = csr_matrix(h_s[i])
    y_test_nonzero_count = y_test_csr.nnz
    if(y_test_nonzero_count == 8):
        c= c +1
print(c)

40000


In [58]:
X_train, X_test, y_train, y_test = train_test_split(Yt_3d, h_s, test_size=0.1, random_state=42)

In [59]:
X_train.shape

(36000, 4096)

In [60]:
y_train.shape

(36000, 4096)

In [61]:
from sklearn.preprocessing import MinMaxScaler
import time
from keras import regularizers
from keras.layers import Dropout

X_train, X_test, y_train, y_test = train_test_split(Yt_3d, h_s, test_size=0.2, random_state=42)

# scaler = MinMaxScaler()
# X_train_normalized = scaler.fit_transform(X_train_reshaped)
# X_test_normalized = scaler.transform(X_test_reshaped)

X_train_reshaped = X_train.reshape(32000,16,16,16)
X_test_reshaped = X_test.reshape(8000,16,16,16)

model = Sequential()

model.add(Conv2D(64, kernel_size=(7,7), input_shape=(16,16,16)))
model.add(LeakyReLU(alpha=0.1))

model.add(Conv2D(32, kernel_size=(5, 5), padding='same'))
model.add(LeakyReLU(alpha=0.1))

model.add(Conv2D(16, kernel_size=(3, 3), padding='same'))
model.add(LeakyReLU(alpha=0.1))

model.add(Conv2D(2, kernel_size=(1, 1)))
model.add(LeakyReLU(alpha=0.1))

model.add(Flatten())
# model.add(Dropout(0.5))  # Adding dropout

model.add(Dense(h_s.shape[1]))  # Corrected placement of kernel_regularizer
model.add(LeakyReLU(alpha=0.1))

model.add(Dense(h_s.shape[1]))  # Corrected placement of kernel_regularizer
model.add(LeakyReLU(alpha=0.1))

custom_optimizer = Adam(learning_rate=0.0005)
model.compile(optimizer=custom_optimizer, loss='mse')

model_checkpoint = ModelCheckpoint('/kaggle/working/best_model.keras', save_best_only=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=10, min_lr=1e-9)

start_time = time.time()
model.fit(X_train_reshaped, y_train, epochs=500, batch_size=64, validation_data=(X_test_reshaped, y_test), callbacks=[model_checkpoint])
end_time = time.time()

model.summary()


/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(
/opt/conda/lib/python3.10/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Epoch 1/500
 28/500 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.0604

W0000 00:00:1714556672.777855     102 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


494/500 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0069

W0000 00:00:1714556676.577124     101 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)               │ (None, 10, 10, 64)     │        50,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_6 (LeakyReLU)       │ (None, 10, 10, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 10, 10, 32)     │        51,232 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_7 (LeakyReLU)       │ (None, 10, 10, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 10, 10, 16)     │         4,624 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_8 (LeakyReLU)       │ (None, 10, 10, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 10, 10, 2)      │            34 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_9 (LeakyReLU)       │ (None, 10, 10, 2)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 200)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 4096)           │       823,296 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_10 (LeakyReLU)      │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 4096)           │    16,781,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_11 (LeakyReLU)      │ (None, 4096)           │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 53,132,216 (202.68 MB)

 Trainable params: 17,710,738 (67.56 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 35,421,478 (135.12 MB)

In [62]:
total_time_seconds = end_time - start_time
total_time_minutes = total_time_seconds / 60

print("Total time taken for training: {:.2f} seconds ({:.2f} minutes)".format(total_time_seconds, total_time_minutes))

Total time taken for training: 1637.52 seconds (27.29 minutes)


In [63]:
model.load_weights('/kaggle/working/best_model.keras')

y_pred = model.predict(X_test_reshaped)

250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step


In [64]:
c = 0
for i in range(0,y_test.shape[0]):
    y_test_csr = csr_matrix(y_test[i])
    y_test_nonzero_count = y_test_csr.nnz
    if(y_test_nonzero_count == 8):
        c= c +1
print(c)

8000


In [65]:
mse = mean_squared_error(y_test, y_pred)
nmse = mse / np.var(y_test)
r = (np.linalg.norm(y_test - y_pred))**2/(np.linalg.norm(y_test))**2
print(f'Normalized Mean Squared Error (NMSE): {nmse}')
print(f'Frobenius Normalized Mean Square Error : {r}')
from sklearn.metrics import r2_score
accuracy = r2_score(y_test, y_pred)
print('Accuracy:', accuracy)

Normalized Mean Squared Error (NMSE): 0.0002649723840709801
Frobenius Normalized Mean Square Error : 0.000264685276338915
Accuracy: 0.06453037080282811


In [66]:
non_zero_indices = np.where(y_test != 0)
y_pred_filtered = y_pred[non_zero_indices]
y_test_filtered = y_test[non_zero_indices]

nmse = (np.linalg.norm(y_test - y_pred)**2 / np.linalg.norm(y_test)**2)
nmse_non_zero = (np.linalg.norm(y_test_filtered - y_pred_filtered)**2 / np.linalg.norm(y_test_filtered)**2)

print("NMSE for All Entries:", nmse)
print("NMSE for Non-Zero Entries:", nmse_non_zero)

NMSE for All Entries: 0.000264685276338915
NMSE for Non-Zero Entries: 0.00015294038796547418


In [67]:
#np.savetxt("ha_pred_25.csv", y_pred, delimiter=",")

In [68]:
y_pred1 = y_pred

In [69]:
threshold = 1e-2
for i in range(y_pred1.shape[0]):
    for j in range(y_pred1.shape[1]):
        if y_pred1[i, j] < threshold:
            y_pred1[i, j] = 0

rmse = np.sqrt(mean_squared_error(y_test, y_pred1))
print('Root Mean Square Error (RMSE):', rmse)

mse = mean_squared_error(y_test, y_pred1)
nmse = mse / np.var(y_test)
r = (np.linalg.norm(y_pred1 - y_test))**2/(np.linalg.norm(y_test))**2
print(f'Normalized Mean Squared Error (NMSE): {nmse}')
print(f'Frobenius Normalized Mean Square Error : {r}')
from sklearn.metrics import r2_score
accuracy = r2_score(y_test, y_pred1)
print('Accuracy:', accuracy)

Root Mean Square Error (RMSE): 0.0007658774030513287
Normalized Mean Squared Error (NMSE): 0.0002330441239912486
Frobenius Normalized Mean Square Error : 0.00023279161175248384
Accuracy: 0.9998466401000732


In [70]:
non_zero_indices = np.where(y_test != 0)
y_pred1_filtered = y_pred1[non_zero_indices]
y_test_filtered = y_test[non_zero_indices]
nmse = (np.linalg.norm(y_test - y_pred1)**2 / np.linalg.norm(y_test)**2)
nmse_non_zero = (np.linalg.norm(y_test_filtered - y_pred1_filtered)**2 / np.linalg.norm(y_test_filtered)**2)

print("NMSE for All Entries:", nmse)
print("NMSE for Non-Zero Entries:", nmse_non_zero)


NMSE for All Entries: 0.00023279161175248384
NMSE for Non-Zero Entries: 0.00015325764921238683


In [71]:
y_pred1.shape

(8000, 4096)

In [72]:
# A_3d.shape

In [73]:
# import numpy as np

# real_part = A_3d['real']
# imag_part = A_3d['imag'] 

# A_3d_numeric = np.column_stack((real_part, imag_part))

# A_3d_numeric = A_3d_numeric.astype(float)

In [74]:
# A_3d_numeric.shape

In [75]:
# psim.shape

In [76]:
# from sklearn.linear_model import OrthogonalMatchingPursuit
# import numpy as np
# omp = OrthogonalMatchingPursuit(n_nonzero_coefs=8)

# # Convert string array to numeric array
# A_3d_numeric = A_3d_numeric.astype(float)

# # y_pred1_numeric = np.array([list(elem) for elem in y_pred1]).astype(float)

# omp.fit(A_3d_numeric, y_pred1.T)

In [77]:
# coeff_real = omp.coef_ 
# support_set = np.where(coeff_real != 0)[0]

# print("\nEstimated Sparse Signal:\n", coeff_real)
# print("\nIndices of Non-Zero Coefficients (Support Set):", support_set)

In [78]:
# coeff_real.shape

In [79]:
# y_test.shape

In [80]:
# rmse = np.sqrt(mean_squared_error(y_test.T, coeff_real))
# print('Root Mean Square Error (RMSE):', rmse)

# mse = mean_squared_error(y_test, coeff_real)
# nmse = mse / np.var(y_test)
# r = (np.linalg.norm(coeff_real - y_test))**2/(np.linalg.norm(y_test))**2
# print(f'Normalized Mean Squared Error (NMSE): {nmse}')
# print(f'Frobenius Normalized Mean Square Error : {r}')
# from sklearn.metrics import r2_score
# accuracy = r2_score(y_test, coeff_real)
# print('Accuracy:', accuracy)

In [81]:
# y_test[1].shape

In [82]:
# y_test_csr = csr_matrix(y_test[4])
# y_test_nonzero_count = y_test_csr.nnz
# print("Number of non-zero values in y_test:", y_test_nonzero_count)

In [83]:
# c = 0
# for i in range(0,8000):
#     y_test_csr = csr_matrix(y_test[i])
#     y_test_nonzero_count = y_test_csr.nnz
#     if(y_test_nonzero_count == 7):
#         c= c +1
# print(c)

In [84]:

y_test_csr = csr_matrix(y_test)
y_pred1_csr = csr_matrix(y_pred1)

# Count non-zero values in each matrix
y_test_nonzero_count = y_test_csr.nnz
y_pred1_nonzero_count = y_pred1_csr.nnz

print("Number of non-zero values in y_test:", y_test_nonzero_count)
print("Number of non-zero values in y_pred1:", y_pred1_nonzero_count)

common_positions = y_test_csr.minimum(y_pred1_csr)

# Count the number of common positions
common_positions_count = common_positions.nnz

print("Number of common positions with non-zero values:", common_positions_count)

Number of non-zero values in y_test: 64000
Number of non-zero values in y_pred1: 67996
Number of common positions with non-zero values: 63776


In [85]:
# y_pred1.to

In [86]:
# Y_total_pred = model.predict(Yt_3d.reshape(40000,16,16,16))

In [87]:
# mse = mean_squared_error(h_s, Y_total_pred)
# nmse = mse / np.var(h_s)
# r = (np.linalg.norm(h_s - Y_total_pred))**2/(np.linalg.norm(h_s))**2
# print(f'Normalized Mean Squared Error (NMSE): {nmse}')
# print(f'Frobenius Normalized Mean Square Error : {r}')
# from sklearn.metrics import r2_score
# accuracy = r2_score(h_s, Y_total_pred)
# print('Accuracy:', accuracy)

In [88]:
# non_zero_indices = np.where(h_s != 0)
# Y_total_pred_filtered = Y_total_pred[non_zero_indices]
# h_s_filtered = h_s[non_zero_indices]

# nmse = (np.linalg.norm(h_s - Y_total_pred)**2 / np.linalg.norm(h_s)**2)
# nmse_non_zero = (np.linalg.norm(h_s_filtered - Y_total_pred_filtered)**2 / np.linalg.norm(h_s_filtered)**2)

# print("NMSE for All Entries:", nmse)
# print("NMSE for Non-Zero Entries:", nmse_non_zero)

In [89]:
# Y_total_pred_1 = Y_total_pred

In [90]:
# threshold = 1e-2
# for i in range(Y_total_pred_1.shape[0]):
#     for j in range(Y_total_pred_1.shape[1]):
#         if Y_total_pred_1[i, j] < threshold:
#             Y_total_pred_1[i, j] = 0

# rmse = np.sqrt(mean_squared_error(h_s, Y_total_pred_1))
# print('Root Mean Square Error (RMSE):', rmse)

# mse = mean_squared_error(h_s, Y_total_pred_1)
# nmse = mse / np.var(h_s)
# r = (np.linalg.norm(Y_total_pred_1 - h_s))**2/(np.linalg.norm(h_s))**2
# print(f'Normalized Mean Squared Error (NMSE): {nmse}')
# print(f'Frobenius Normalized Mean Square Error : {r}')
# from sklearn.metrics import r2_score
# accuracy = r2_score(h_s, Y_total_pred_1)
# print('Accuracy:', accuracy)

In [91]:
# non_zero_indices = np.where(h_s != 0)
# Y_total_pred_1_filtered = Y_total_pred_1[non_zero_indices]
# h_s_filtered = h_s[non_zero_indices]
# nmse = (np.linalg.norm(h_s - Y_total_pred_1)**2 / np.linalg.norm(h_s)**2)
# nmse_non_zero = (np.linalg.norm(h_s_filtered - Y_total_pred_1_filtered)**2 / np.linalg.norm(h_s_filtered)**2)

# print("NMSE for All Entries:", nmse)
# print("NMSE for Non-Zero Entries:", nmse_non_zero)

In [92]:
3# from scipy.sparse import csr_matrix

# y_test_csr = csr_matrix(y_test)
# y_pred1_csr = csr_matrix(y_pred)

# # Count non-zero values in each matrix
# y_test_nonzero_count = y_test_csr.nnz
# y_pred1_nonzero_count = y_pred1_csr.nnz

# print("Number of non-zero values in y_test:", y_test_nonzero_count)
# print("Number of non-zero values in y_pred1:", y_pred1_nonzero_count)

# common_positions = y_test_csr.minimum(y_pred1_csr)

# # Count the number of common positions
# common_positions_count = common_positions.nnz

# print("Number of common positions with non-zero values:", common_positions_count)

3